In [62]:
class RS:
    def __init__(self, s, function):
        self.s = s  
        self.function = function 

    def gen(self):
        output = self.s[len(self.s) - 1]  
        value = 0  
        for i in self.function:  
            value += self.s[i]
        value %= 2
        self.s = [value] + self.s[:len(self.s) - 1]  # Сдвиг
        return output

from random import randint

def to_bin(a,l):
    result = []
    for i in range(l):
        result.append(a % 2)
        a = a // 2
    result.reverse()
    return result

def from_bin(a):
    result = 0
    for i in range(len(a)):
        result += a[i] * 2 ** (len(a) - i - 1)
    return result
    
from tqdm import tqdm
import time
import scipy.stats as stats
beta = 0.0000001
def attack_reg(n,rr):
    count =0
    for i in range(len(f)):
        if x[n][i]==f[i]:
            count+=1
    q0=1/2
    q1=count/len(f)
    if q1!=1/2:
        quantil = stats.norm.ppf(1-beta)
        N = ((quantil * (q0 * (1 - q0)) ** (1 / 2) + quantil * (q1 * (1 - q1)) ** (1 / 2)) ** 2) / (q0 - q1) ** 2
        N=int(N)+1
        C = quantil * (N * q1 * (1 - q1)) ** (1 / 2) + N * (1-q1)
        C=int(C)
        print('-'*5+f'регистр {n+1}'+'-'*5)
        print(f'N={N} C={C}')
        regs = []
        for i in range(rr):
            regs.append(RS(start[i],xor[i]))
        gen_out = []
        for i in range(N):
            v=[]
            for j in range(rr):
                v.append(regs[j].gen())
            gen_out.append(f[from_bin(v)])
        mb = []
        for i in tqdm(range(2**len(start[n]))):
            reg = RS(to_bin(i,len(start[n])),xor[n])
            c=0
            for j in range(N):
               if reg.gen() != gen_out[j]:
                   c+=1
               if c>C:
                   break
            if c<=C:
               mb.append((i,c))
        
        for i in mb:
            print(f'Кандидат:{i[0]} С={i[1]}')
        print()
        return [i[0] for i in mb]        
    else:
        return [i for i in range(2**len(start[n]))]
        
def attack(n_regs):
    total = []
    for i in range(n_regs):
        print(f'начальное заполнение регистра r{i+1}=',end='')
        for j in range(len(start[i])):
            print(f'{start[i][j]}',end='')
        print(f' {from_bin(start[i])}')
    for i in range(n_regs):
        total.append(attack_reg(i,n_regs))
    return total

# Пример с занятия

In [63]:
x=[
    [0]*4+[1]*4,
    ([0]*2+[1]*2)*2,
    [0,1]*4
]
start = [
    [1,1,1],
    [1,1],
    [1,1]
]
xor = [
    [1,2],
    [1,0],
    [1,0]
]
f=[0]*7+[1]
attack(3)

начальное заполнение регистра r1=111 7
начальное заполнение регистра r2=11 3
начальное заполнение регистра r3=11 3
-----регистр 1-----
N=1676 C=731


100%|███████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 999.15it/s]


Кандидат:0 С=639
Кандидат:7 С=320

-----регистр 2-----
N=1676 C=731


100%|██████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 1337.26it/s]


Кандидат:0 С=639
Кандидат:3 С=479

-----регистр 3-----
N=1676 C=731


100%|██████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 1002.64it/s]

Кандидат:0 С=639
Кандидат:3 С=479



[[0, 7], [0, 3], [0, 3]]

# Для курсовой

In [64]:
x=[
    [0]*32+[1]*32,
    ([0]*16+[1]*16)*2,
    ([0]*8+[1]*8)*4,
    ([0]*4+[1]*4)*8,
    ([0]*2+[1]*2)*16,
    [0,1]*32
]
start = [
    [randint(0,1) for i in range(13)],
    [randint(0,1) for i in range(13)],
    [randint(0,1) for i in range(13)],
    [randint(0,1) for i in range(13)],
    [randint(0,1) for i in range(13)],
    [randint(0,1) for i in range(15)]
]
xor = [
    [12,11,9,8], # x^13+x^4+x^3+x+1
    [12,10,9,8], # x^13+x^4+x^3+x^2+1
    [12,11,10,9,8], #x^13+x^4+x^3+x^2+x+1
    [12,11,10,8], #x^13+x^4+x^2+x+1
    [12,9], #x^13+x^3+1
    [14,13] #x^15+x+1
]

f="0001000111010100000011111001101100111000010111111011100010000111"
f=[int(ch) for ch in f]
total=attack(6)

начальное заполнение регистра r1=0101111100001 3041
начальное заполнение регистра r2=1111010110010 7858
начальное заполнение регистра r3=1000111011001 4569
начальное заполнение регистра r4=0001101101010 874
начальное заполнение регистра r5=0001000110101 565
начальное заполнение регистра r6=001110111011100 7644
-----регистр 1-----
N=27628 C=13381


100%|██████████████████████████████████████████████████████████████████████████████| 8192/8192 [02:26<00:00, 55.89it/s]


Кандидат:3041 С=12963

-----регистр 2-----
N=27628 C=13381


100%|██████████████████████████████████████████████████████████████████████████████| 8192/8192 [02:26<00:00, 55.85it/s]


Кандидат:7858 С=12893

-----регистр 3-----
N=3022 C=1368


...

Кандидат:874 С=3044
Кандидат:5947 С=3201

-----регистр 5-----
N=27628 C=13381


100%|██████████████████████████████████████████████████████████████████████████████| 8192/8192 [02:03<00:00, 66.08it/s]


Кандидат:565 С=12887

-----регистр 6-----
N=27628 C=13381


100%|████████████████████████████████████████████████████████████████████████████| 32768/32768 [08:23<00:00, 65.02it/s]

Кандидат:7644 С=12985



# Перебор кандидатов

In [66]:
total

[[3041], [7858], [4569], [874, 5947], [565], [7644]]

In [92]:
regs=[]
for i in range(6):
    regs.append(RS(start[i],xor[i]))
gen_out = []
for i in range(27628):
    v=[j.gen() for j in regs]
    gen_out.append(f[from_bin(v)])

for r0 in total[0]:
    for r1 in total[1]:
        for r2 in total[2]:
            for r3 in total[3]:
                for r4 in total[4]:
                    for r5 in total[5]:
                        out=[]
                        regs = [
                            RS(to_bin(r0,13),xor[0]),
                            RS(to_bin(r1,13),xor[1]),
                            RS(to_bin(r2,13),xor[2]),
                            RS(to_bin(r3,13),xor[3]),
                            RS(to_bin(r4,13),xor[4]),
                            RS(to_bin(r5,15),xor[5])
                        ]
                        for i in range(27628):
                            v=[j.gen() for j in regs]
                            out.append(f[from_bin(v)])
                        if out == gen_out:
                            st=''
                            st+=f'r1={r0}\n'
                            st+=f'r2={r1}\n'
                            st+=f'r3={r2}\n'
                            st+=f'r4={r3}\n'
                            st+=f'r5={r4}\n'
                            st+=f'r6={r5}\n'
                            print(st)

r1=3041
r2=7858
r3=4569
r4=874
r5=565
r6=7644

